<a href="https://colab.research.google.com/github/JunShimura/DXI2024Problem01/blob/main/problem_01_Week4_rollback_105_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[M5 Forecasting - Accuracy Competition](https://www.kaggle.com/c/m5-forecasting-accuracy)


SetUp

In [ ]:
! nvidia-smi
import tensorflow as tf
print(f"GPU for tensowflow:{tf.test.gpu_device_name()}");

/bin/bash: line 1: nvidia-smi: command not found
GPU for tensowflow:


## enable GPU >> 没

In [ ]:
# !apt install --no-install-recommends cmake build-essential clinfo opencl-headers libboost-all-dev
# !pip install -U pip setuptools wheel
# !pip install numpy scipy scikit-learn
# !mkdir -p /etc/OpenCL/vendors && echo "libnvidia-opencl.so.1" > /etc/OpenCL/vendors/nvidia.icd
# !pip install lightgbm

In [ ]:
import lightgbm as lgb
print(f"LGB version:{lgb.__version__}")
print(lgb.basic._ConfigAliases.get("gpu_device_id"))

/usr/local/lib/python3.11/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


LGB version:4.5.0
{'gpu_device_id'}


## BASIC SETUP

In [ ]:
# Connect GoogleDrive to mount using stored token
from google.colab import drive
import os
import json

drive.mount("/content/drive", force_remount=True)
fin = open("/content/drive/MyDrive/kaggle/token/kaggle.json", "r")
json_data = json.load(fin)
fin.close()
os.environ["KAGGLE_USERNAME"] = json_data["username"]
os.environ["KAGGLE_KEY"] = json_data["key"]

Mounted at /content/drive


In [ ]:
# Download Competition data
%%bash
kaggle competitions download -c m5-forecasting-accuracy --force
if [ $? -ne 0 ]; then
    echo "データのダウンロードに問題があったようです．"
    echo "競技規則に同意し，APIキーが有効であることを確認してください．"
else
    mkdir -p /content/kaggle
    unzip -o /content/m5-forecasting-accuracy.zip -d /content/kaggle
fi
wget -q -P /tmp https://noto-website-2.storage.googleapis.com/pkgs/NotoSansCJKjp-hinted.zip
unzip -u /tmp/NotoSansCJKjp-hinted.zip -d /usr/share/fonts/NotoSansCJKjp


Archive:  /content/m5-forecasting-accuracy.zip
  inflating: /content/kaggle/calendar.csv  
  inflating: /content/kaggle/sales_train_evaluation.csv  
  inflating: /content/kaggle/sales_train_validation.csv  
  inflating: /content/kaggle/sample_submission.csv  
  inflating: /content/kaggle/sell_prices.csv  
Archive:  /tmp/NotoSansCJKjp-hinted.zip
  inflating: /usr/share/fonts/NotoSansCJKjp/LICENSE_OFL.txt  
  inflating: /usr/share/fonts/NotoSansCJKjp/NotoSansCJKjp-Black.otf  
  inflating: /usr/share/fonts/NotoSansCJKjp/NotoSansCJKjp-Bold.otf  
  inflating: /usr/share/fonts/NotoSansCJKjp/NotoSansCJKjp-DemiLight.otf  
  inflating: /usr/share/fonts/NotoSansCJKjp/NotoSansCJKjp-Light.otf  
  inflating: /usr/share/fonts/NotoSansCJKjp/NotoSansCJKjp-Medium.otf  
  inflating: /usr/share/fonts/NotoSansCJKjp/NotoSansCJKjp-Regular.otf  
  inflating: /usr/share/fonts/NotoSansCJKjp/NotoSansCJKjp-Thin.otf  
  inflating: /usr/share/fonts/NotoSansCJKjp/NotoSansMonoCJKjp-Bold.otf  
  inflating: /usr/shar

100%|██████████| 45.8M/45.8M [00:02<00:00, 18.4MB/s]


In [ ]:
# Python enviroment
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.seasonal import STL
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.forecasting.stl import STLForecast
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import gc  # ガベージコレクションを明示的に呼び出す

font_path = '/usr/share/fonts/NotoSansCJKjp/NotoSansMonoCJKjp-Regular.otf'
matplotlib.font_manager.fontManager.addfont(font_path)
prop = matplotlib.font_manager.FontProperties(fname=font_path)
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = prop.get_name()
os.chdir('/content/kaggle')

In [ ]:
# load sales
sales = pd.read_csv('sales_train_evaluation.csv')
sales.head(10)

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,4,0,0,0,0,3,3,0,1
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,1,2,1,1,0,0,0,0,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,2,0,0,0,2,3,0,1
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,1,0,4,0,1,3,0,2,6
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,2,1,0,0,2,1,0
5,HOBBIES_1_006_CA_1_evaluation,HOBBIES_1_006,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,0,0,1,0,0,5,2,0
6,HOBBIES_1_007_CA_1_evaluation,HOBBIES_1_007,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,1,0,0,0,1,0,1,1,0
7,HOBBIES_1_008_CA_1_evaluation,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,12,15,0,0,...,7,0,6,0,15,5,4,1,40,32
8,HOBBIES_1_009_CA_1_evaluation,HOBBIES_1_009,HOBBIES_1,HOBBIES,CA_1,CA,2,0,7,3,...,1,0,0,0,0,0,0,0,1,0
9,HOBBIES_1_010_CA_1_evaluation,HOBBIES_1_010,HOBBIES_1,HOBBIES,CA_1,CA,0,0,1,0,...,0,0,1,0,2,1,1,0,0,1


In [ ]:
# load calender
calendar = pd.read_csv('calendar.csv')
calendar

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1964,2016-06-15,11620,Wednesday,5,6,2016,d_1965,NaN,NaN,NaN,NaN,0,1,1
1965,2016-06-16,11620,Thursday,6,6,2016,d_1966,NaN,NaN,NaN,NaN,0,0,0
1966,2016-06-17,11620,Friday,7,6,2016,d_1967,NaN,NaN,NaN,NaN,0,0,0
1967,2016-06-18,11621,Saturday,1,6,2016,d_1968,NaN,NaN,NaN,NaN,0,0,0


In [ ]:
# load sell prices
# sell_prices = pd.read_csv('sell_prices.csv')
# sell_prices.head()

In [ ]:
# load Submission
submission = pd.read_csv('sample_submission.csv')
submission.shape


(60980, 29)

In [ ]:
submission_ids = submission['id'].unique().sort()

# salesのidのユニークな要素
unique_sales_ids = sales['id'].unique().sort()

# 全ての要素がsubmissionのidに含まれているか確認
print(np.array_equal( submission_ids,unique_sales_ids))
#all_ids_match = all(elem in submission['id'].values for elem in unique_sales_ids)
#print(f"All ids in sales match submission ids: {all_ids_match}")

# submissionのidを数値にマッピング
id_map = {id_value: idx for idx, id_value in enumerate(submission['id'])}
del submission


True


# データの分析

* 商品は発売日前の売上が0になっている様子
* 週、一ヶ月で周期性が見られる
* 年次イベントでスパイクがある
* 店ごとで地域の災害による影響がある
* 全体で僅かな正の相関が見られる

## [参考]
* 日本語解説
 *  https://www.kaggle.com/code/junshimura/back-to-predict-the-future-m5-eda-6-1
 * https://www.kaggle.com/code/takahiro1127/starter-data-exploration-lstm



# データ準備

## 基本手順
1. 商品のIDをキーにして日付、曜日、店、地域、販売数の形にpivot
1. 日付の列を変更
  1. 学習データの変更
    1. 日付をint型にする
    1. 日付のYYYYMMDD形式に分解、それぞれを列にする
    1. WeekDayをカレンダーからキーで付与
    1. 日付のシリアル値・年・月・日・曜日
  1. カレンダーの変更
    1. 日付のYYYYMMDD形式に分解、それぞれを列にする
1. カレンダーのデータをシリアル値をキーにtrainデータと併合する



In [ ]:
# 商品のIDをキーにして日付、曜日、店、地域、販売数の形にpivot
sales_modeling = sales.melt(id_vars=['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], var_name='day', value_name='sales')
del sales
sales_modeling
# gemini recomended
# melt関数でデータを長い形式に変換
# df_melt = pd.melt(df, id_vars=['id', 'item_id'], var_name='dayNo', value_name='amount')

# Dr.Yamada
# sales_modeling = sales.groupby('cat_id', as_index=False).agg({col: 'sum' for col in sales.columns if col.startswith('d_')})
# sales_modeling = sales_modeling.melt(id_vars=['cat_id'], var_name='day', value_name='sales')
# sales_modeling['day'] = sales_modeling['day'].str.replace('d_', '').astype(int)
# sales_modeling = sales_modeling.pivot(index='day', columns='cat_id', values='sales')
# sales_modeling = sales_modeling.sort_index()
# sales_modeling = sales_modeling.set_index(pd.to_datetime(pd.read_csv('calendar.csv')['date'].values[:1941]))
# sales_modeling.index.freq = 'D'


,id,item_id,dept_id,cat_id,store_id,state_id,day,sales
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
...,...,...,...,...,...,...,...,...
59181085,FOODS_3_823_WI_3_evaluation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,d_1941,1
59181086,FOODS_3_824_WI_3_evaluation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,d_1941,0
59181087,FOODS_3_825_WI_3_evaluation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,d_1941,2
59181088,FOODS_3_826_WI_3_evaluation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,d_1941,0


In [ ]:
# 文字列を数値化,Drop
def factorize(df, column):
    if column in df.columns:
      df[column + '_enc'], uniques = pd.factorize(df[column])
      df.drop(column, axis=1, inplace=True)
      df.rename(columns={column+'_enc': column}, inplace=True)
      if uniques.size<255:
         df[column]=df[column].astype(np.int8)
      print(f"REPLAED {column} {uniques.size } uniqes")
      return df, uniques
    else:
      print("NOT FOUND ", column)
      return df, []

# salesのid列を可逆的に数値化
sales_modeling['encoded_id'] = sales_modeling['id'].map(id_map)
sales_modeling.drop('id', axis=1, inplace=True)
sales_modeling.rename(columns={'encoded_id': 'id'}, inplace=True)

sales_modeling['d'] = sales_modeling['day'].str.replace('d_', '').astype(int)
sales_modeling.drop('day', axis=1, inplace=True)

sales_modeling, item_uniques = factorize(sales_modeling,'item_id')
sales_modeling, dept_uniques = factorize(sales_modeling,'dept_id')
sales_modeling, cat_uniques = factorize(sales_modeling,'cat_id')
sales_modeling, store_uniques = factorize(sales_modeling,'store_id')
sales_modeling, state_uniques = factorize(sales_modeling,'state_id')

def onehot(df, column):
    one_hot = pd.get_dummies(df[column], prefix=column).astype(np.int8)  # True/False を 1/0 に変換
    df.drop(column, axis=1, inplace=True)
    df = pd.concat([df, one_hot], axis=1)
    return df

sales_modeling = onehot(sales_modeling,'cat_id')
sales_modeling = onehot(sales_modeling,'dept_id')
sales_modeling = onehot(sales_modeling,'store_id')
sales_modeling


REPLAED item_id 3049 uniqes
REPLAED dept_id 7 uniqes
REPLAED cat_id 3 uniqes
REPLAED store_id 10 uniqes
REPLAED state_id 3 uniqes


,sales,id,d,item_id,state_id,cat_id_0,cat_id_1,cat_id_2,dept_id_0,dept_id_1,...,store_id_0,store_id_1,store_id_2,store_id_3,store_id_4,store_id_5,store_id_6,store_id_7,store_id_8,store_id_9
0,0,30490,1,0,0,1,0,0,1,0,...,1,0,0,0,0,0,0,0,0,0
1,0,30491,1,1,0,1,0,0,1,0,...,1,0,0,0,0,0,0,0,0,0
2,0,30492,1,2,0,1,0,0,1,0,...,1,0,0,0,0,0,0,0,0,0
3,0,30493,1,3,0,1,0,0,1,0,...,1,0,0,0,0,0,0,0,0,0
4,0,30494,1,4,0,1,0,0,1,0,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59181085,1,60975,1941,3044,2,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
59181086,0,60976,1941,3045,2,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
59181087,2,60977,1941,3046,2,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
59181088,0,60978,1941,3047,2,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1


In [ ]:
# Read calendar
calendar = pd.read_csv('calendar.csv')
calendar_enc =calendar
print(calendar_enc["wday"].unique())
print(calendar_enc["weekday"].unique())
# Encord strings
calendar_enc['d'] = calendar['d'].str.replace('d_', '').astype(int)
calendar_enc, event_name_1_uniqes= factorize( calendar_enc,'event_name_1')
calendar_enc, event_type_1_uniqes= factorize( calendar_enc,'event_type_1')
calendar_enc, event_name_2_uniqes= factorize( calendar_enc,'event_name_2')
calendar_enc, event_type_2_uniqes= factorize( calendar_enc,'event_type_2')

calendar_enc["dd"] = pd.to_datetime(calendar_enc["date"]).dt.day.astype("int8")
calendar_enc.drop('wm_yr_wk', axis=1, inplace=True)
calendar_enc.drop('weekday', axis=1, inplace=True)
calendar_enc.drop('month', axis=1, inplace=True)
calendar_enc.drop('year', axis=1, inplace=True)

def replace_event_labels(df):
    event_columns = ["event_name_1", "event_type_1"]
    df[event_columns] = df[event_columns].applymap(lambda x: 0 if x == -1 else 1).astype(np.int8)
    return df


# ワンホットエンコーディング
# calendar_enc['wday_1'] = np.where(calendar_enc['wday']==1,1,0).astype(np.int8)
# calendar_enc['wday_2'] = np.where(calendar_enc['wday']==2,1,0).astype(np.int8)
# calendar_enc.drop('wday', axis=1, inplace=True)
calendar_enc = onehot(calendar_enc,'wday')
# calendar_enc = onehot(calendar_enc,'event_name_1')
calendar_enc = replace_event_labels(calendar_enc)


# Data cache
drive.mount('/content/drive')
calendar_enc.to_csv('/content/drive/My Drive/calendar_enc.csv', index=False)
calendar_enc


[1 2 3 4 5 6 7]
['Saturday' 'Sunday' 'Monday' 'Tuesday' 'Wednesday' 'Thursday' 'Friday']
REPLAED event_name_1 30 uniqes
REPLAED event_type_1 4 uniqes
REPLAED event_name_2 4 uniqes
REPLAED event_type_2 2 uniqes


<ipython-input-14-a91bcd74fa6e>:21: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[event_columns] = df[event_columns].applymap(lambda x: 0 if x == -1 else 1).astype(np.int8)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,date,d,snap_CA,snap_TX,snap_WI,event_name_1,event_type_1,event_name_2,event_type_2,dd,wday_1,wday_2,wday_3,wday_4,wday_5,wday_6,wday_7
0,2011-01-29,1,0,0,0,0,0,-1,-1,29,1,0,0,0,0,0,0
1,2011-01-30,2,0,0,0,0,0,-1,-1,30,0,1,0,0,0,0,0
2,2011-01-31,3,0,0,0,0,0,-1,-1,31,0,0,1,0,0,0,0
3,2011-02-01,4,1,1,0,0,0,-1,-1,1,0,0,0,1,0,0,0
4,2011-02-02,5,1,0,1,0,0,-1,-1,2,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1964,2016-06-15,1965,0,1,1,0,0,-1,-1,15,0,0,0,0,1,0,0
1965,2016-06-16,1966,0,0,0,0,0,-1,-1,16,0,0,0,0,0,1,0
1966,2016-06-17,1967,0,0,0,0,0,-1,-1,17,0,0,0,0,0,0,1
1967,2016-06-18,1968,0,0,0,0,0,-1,-1,18,1,0,0,0,0,0,0


In [ ]:
# カレンダーデータを sales_data に結合

merged_data = pd.merge(sales_modeling, calendar_enc, on='d', how='left')

del sales_modeling
del calendar_enc
gc.collect()



31

### >>>データの一時保存・読み込み

In [ ]:
# Data cache
#drive.mount('/content/drive')
#merged_data.to_csv('/content/drive/My Drive/merged_data.csv', index=False)
# データの読み込み
# df = pd.read_csv('/content/drive/My Drive/merged_data.csv')

#学習

## 検討項目

* モデルの選択
* 店舗・商品毎にモデルを変える必要性の検討



In [ ]:
df = merged_data

## 日付の変換・特徴量を追加

In [ ]:

# 日付の型変換
df['date'] = pd.to_datetime(df['date'])

# 特徴量エンジニアリング例
df['lag_7'] = df.groupby('id')['sales'].shift(7)  # 過去7日間の売上
df['lag_28'] = df.groupby('id')['sales'].shift(28)  # 過去28日間の売上
df['rolling_mean_7'] = df.groupby('id')['sales'].transform(lambda x: x.shift(1).rolling(7).mean())
df['rolling_mean_28'] = df.groupby('id')['sales'].transform(lambda x: x.shift(1).rolling(28).mean())

# 欠損値の処理
df = df.dropna()



In [ ]:

# メモリ最適化関数
def reduce_memory_usage(df):
    for col in df.columns:
        col_type = df[col].dtype

        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()

            if str(col_type).startswith('int'):
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                else:
                    df[col] = df[col].astype(np.int64)
            elif str(col_type).startswith('float'):
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    return df

# 例：データを最適化
df = reduce_memory_usage(df)


<ipython-input-19-274b19420bc4>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].astype(np.int16)
<ipython-input-19-274b19420bc4>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].astype(np.int32)
<ipython-input-19-274b19420bc4>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide

## 訓練と評価

In [ ]:
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# 訓練データとターゲットの分割
features = ['d','dd',
            'lag_7', 'lag_28', 'rolling_mean_7', 'rolling_mean_28',
            'snap_CA', 'snap_TX', 'snap_WI', 'event_name_1', 'event_type_1',
            'item_id',
            'dept_id_0',  'dept_id_1', 'dept_id_2', 'dept_id_3', 'dept_id_4', 'dept_id_5', 'dept_id_6',
            'cat_id_0', 'cat_id_1', 'cat_id_2',
            'store_id_0', 'store_id_1', 'store_id_2', 'store_id_3', 'store_id_4', 'store_id_5', 'store_id_6', 'store_id_7','store_id_8', 'store_id_9',
            'state_id', 'wday_1', 'wday_2', 'wday_3', 'wday_4', 'wday_5', 'wday_6', 'wday_7']

X = df[features] # メモリ効率向上のため float32 に変換
y = df['sales'].astype(np.float32)

# 訓練データとテストデータの分割（shuffle=False にしてシャッフル処理を省略）
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42, shuffle=False)

# LightGBM 用データセット作成（free_raw_data=False でデータ削減）
lgb_train = lgb.Dataset(X_train, y_train, free_raw_data=True)
lgb_test = lgb.Dataset(X_test, y_test, reference=lgb_train, free_raw_data=True)

# LightGBM パラメータ設定（並列処理 & GPU 最適化）
params = {
    'objective': 'regression',
    'metric': 'rmse',
    'verbosity': -1,
    'num_leaves': 16,  # デフォルトは 31
    'max_depth': -1,   # 制限なし → 小さくするとメモリ削減
    'num_threads': -1,  # 全 CPU コアを使用
    # 'device_type': 'gpu',  # GPU がある場合に使用
    'feature_pre_filter': False,  # 事前フィルタリング無効化で高速化
    'early_stopping_rounds': 10,  # 早期停止をここで指定
    'max_bin': 32  # メモリ節約のためビンサイズを減らす
}

params['bin_construct_sample_cnt'] = 500000  # デフォルト: 2000000


# モデルの訓練
model = lgb.train(params, lgb_train, valid_sets=[lgb_train, lgb_test])

# 予測
y_pred = model.predict(X_test)

# 評価（mean_squared_error で直接 RMSE を計算）
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f'RMSE: {rmse}')

del X_train, X_test, y_train, y_test, lgb_train, lgb_test, params, y_pred, rmse
gc.collect()


RMSE: 2.086819694482078


101

## 予測データを構築

In [ ]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from joblib import Parallel, delayed
from sklearn.preprocessing import LabelEncoder

# カテゴリ変数をエンコードするための辞書
label_encoders = {}

def process_product(product_id, product_data, calendar_dict):
    """各商品 ID に対する 28 日間の予測処理"""
    latest_data = product_data.copy()

    # 商品情報
    item_id = latest_data['item_id'].values[0]
    dept_id_0 = latest_data['dept_id_0'].values[0]
    dept_id_1 = latest_data['dept_id_1'].values[0]
    dept_id_2 = latest_data['dept_id_2'].values[0]
    dept_id_3 = latest_data['dept_id_3'].values[0]
    dept_id_4 = latest_data['dept_id_4'].values[0]
    dept_id_5 = latest_data['dept_id_5'].values[0]
    dept_id_6 = latest_data['dept_id_6'].values[0]
    cat_id_0 = latest_data['cat_id_0'].values[0]
    cat_id_1 = latest_data['cat_id_1'].values[0]
    cat_id_2 = latest_data['cat_id_2'].values[0]
    store_id_0 = latest_data['store_id_0'].values[0]
    store_id_1 = latest_data['store_id_1'].values[0]
    store_id_2 = latest_data['store_id_2'].values[0]
    store_id_3 = latest_data['store_id_3'].values[0]
    store_id_4 = latest_data['store_id_4'].values[0]
    store_id_5 = latest_data['store_id_5'].values[0]
    store_id_6 = latest_data['store_id_6'].values[0]
    store_id_7 = latest_data['store_id_7'].values[0]
    store_id_8 = latest_data['store_id_8'].values[0]
    store_id_9 = latest_data['store_id_9'].values[0]
    state_id = latest_data['state_id'].values[0]

    product_predictions = []

    for day in range(1, future_days + 1):
        next_date = last_date + pd.Timedelta(days=day)
        d_value = max_d + day

        # `calendar_dict` からデータを取得 (O(1) アクセス)
        calendar_row = calendar_dict.get(d_value, {
            'dd':0,'wday_1':0,'wday_2':0,'wday_3':0,'wday_4':0,'wday_5':0,'wday_6':0,'wday_7':0,
            'snap_CA': 0, 'snap_TX': 0, 'snap_WI': 0,
            'event_name_1': 0, 'event_type_1': 0,
        })

        # 特徴量を計算
        lag_7 = product_data['sales'].iloc[-7:].mean() if len(product_data) >= 7 else np.nan
        lag_28 = product_data['sales'].iloc[-28:].mean() if len(product_data) >= 28 else np.nan
        rolling_mean_7 = product_data['sales'].iloc[-7:].mean() if len(product_data) >= 7 else np.nan
        rolling_mean_28 = product_data['sales'].iloc[-28:].mean() if len(product_data) >= 28 else np.nan

        # 予測データ作成
        future_row = {
            'd': d_value,
            'dd': calendar_row['dd'],
            'lag_7': lag_7,
            'lag_28': lag_28,
            'rolling_mean_7': rolling_mean_7,
            'rolling_mean_28': rolling_mean_28,
            'wday_1': calendar_row['wday_1'],
            'wday_2': calendar_row['wday_2'],
            'wday_3': calendar_row['wday_3'],
            'wday_4': calendar_row['wday_4'],
            'wday_5': calendar_row['wday_5'],
            'wday_6': calendar_row['wday_6'],
            'wday_7': calendar_row['wday_7'],
            'snap_CA': calendar_row['snap_CA'],
            'snap_TX': calendar_row['snap_TX'],
            'snap_WI': calendar_row['snap_WI'],
            'event_name_1': calendar_row['event_name_1'],
            'event_type_1': calendar_row['event_type_1'],
            'item_id': item_id,
            'dept_id_0': dept_id_0,
            'dept_id_1': dept_id_1,
            'dept_id_2': dept_id_2,
            'dept_id_3': dept_id_3,
            'dept_id_4': dept_id_4,
            'dept_id_5': dept_id_5,
            'dept_id_6': dept_id_6,
            'cat_id_0': cat_id_0,
            'cat_id_1': cat_id_1,
            'cat_id_2': cat_id_2,
            'store_id_0': store_id_0,
            'store_id_1': store_id_1,
            'store_id_2': store_id_2,
            'store_id_3': store_id_3,
            'store_id_4': store_id_4,
            'store_id_5': store_id_5,
            'store_id_6': store_id_6,
            'store_id_7': store_id_7,
            'store_id_8': store_id_8,
            'store_id_9': store_id_9,
            'state_id': state_id,

        }

        # モデルで予測
        future_sales = model.predict(pd.DataFrame([future_row]))[0]
        future_row['sales'] = future_sales
        future_row['date'] = next_date
        future_row['id'] = product_id

        product_predictions.append(future_row)

    return product_predictions


# データの読み込み
calendar_enc = pd.read_csv('/content/drive/My Drive/calendar_enc.csv')

future_days = 28
ids = df['id'].unique()
last_date = df['date'].max()
max_d = df['d'].max()

# `calendar_enc` を辞書化（numpy 型を Python ネイティブ型に変換）
calendar_dict = {
    int(row['d']): {
        'dd': int(row['dd']),
        'wday_1': int(row['wday_1']),
        'wday_2': int(row['wday_2']),
        'wday_3': int(row['wday_3']),
        'wday_4': int(row['wday_4']),
        'wday_5': int(row['wday_5']),
        'wday_6': int(row['wday_6']),
        'wday_7': int(row['wday_7']),
        'snap_CA': int(row['snap_CA']),
        'snap_TX': int(row['snap_TX']),
        'snap_WI': int(row['snap_WI']),
        'event_name_1': int(row['event_name_1']) if pd.notna(row['event_name_1']) else 0,
        'event_type_1': int(row['event_type_1']) if pd.notna(row['event_type_1']) else 0,
    }
    for _, row in calendar_enc.iterrows()
}

# 商品ごとのデータを辞書に変換
product_groups = {pid: group for pid, group in df.groupby('id')}

# tqdm の外部管理
with tqdm(total=len(product_groups), desc="Processing products") as pbar:
    results = Parallel(n_jobs=-1, prefer="processes")(
    delayed(process_product)(pid, data, calendar_dict) for pid, data in product_groups.items()
)
pbar.update(len(product_groups))  # tqdm の更新を外部で実施

# 予測結果を DataFrame に変換
future_predictions = [row for sublist in results for row in sublist]
future_df = pd.DataFrame(future_predictions)

print(future_df.head(28))


Processing products:   0%|          | 0/30490 [08:49<?, ?it/s]


       d  dd  lag_7    lag_28  rolling_mean_7  rolling_mean_28  wday_1  \
0   1942  23    1.0  1.178571             1.0         1.178571       0   
1   1943  24    1.0  1.178571             1.0         1.178571       0   
2   1944  25    1.0  1.178571             1.0         1.178571       0   
3   1945  26    1.0  1.178571             1.0         1.178571       0   
4   1946  27    1.0  1.178571             1.0         1.178571       0   
5   1947  28    1.0  1.178571             1.0         1.178571       1   
6   1948  29    1.0  1.178571             1.0         1.178571       0   
7   1949  30    1.0  1.178571             1.0         1.178571       0   
8   1950  31    1.0  1.178571             1.0         1.178571       0   
9   1951   1    1.0  1.178571             1.0         1.178571       0   
10  1952   2    1.0  1.178571             1.0         1.178571       0   
11  1953   3    1.0  1.178571             1.0         1.178571       0   
12  1954   4    1.0  1.178571         

## 予測データを保存

In [ ]:
# prompt: only d sales id write to csv in googldDrive

# Assuming future_df is the DataFrame created in the previous code

# Select only the 'id', 'd', and 'sales' columns
sales_data = future_df[['id', 'd', 'sales']]

# Specify the file path in Google Drive
# file_path = '/content/drive/My Drive/sales_predictions.csv'

# Write the data to a CSV file
# sales_data.to_csv(file_path, index=False)

# print(f"Sales predictions for 'd' saved to {file_path}")

# データの読み込み
# sales_data = pd.read_csv('/content/drive/My Drive/sales_predictions.csv')

### データの一時保存・読み込み

# 提出

In [ ]:
sales_data

,id,d,sales
0,30490,1942,0.997321
1,30490,1943,0.980085
2,30490,1944,0.998054
3,30490,1945,0.998054
4,30490,1946,0.998054
...,...,...,...
853715,60979,1965,1.844649
853716,60979,1966,1.838835
853717,60979,1967,1.834941
853718,60979,1968,1.834941


In [ ]:
# load Submission
submission = pd.read_csv('sample_submission.csv')
submission

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,HOBBIES_1_002_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,HOBBIES_1_004_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,HOBBIES_1_005_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60975,FOODS_3_823_WI_3_evaluation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
60976,FOODS_3_824_WI_3_evaluation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
60977,FOODS_3_825_WI_3_evaluation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
60978,FOODS_3_826_WI_3_evaluation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# submissionのidを数値にマッピング
id_map = {id_value: idx for idx, id_value in enumerate(submission['id'])}
print(len(id_map))


60980


In [ ]:
# 逆引き辞書を作成
reverse_map = {v: k for k, v in id_map.items()}

In [ ]:
# idをマッピング
# id_map を使って sales_data の id を文字列に変換
sales_data['id_str'] = sales_data['id'].map(reverse_map)
sales_data


<ipython-input-27-a8a1bb973689>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sales_data['id_str'] = sales_data['id'].map(reverse_map)


,id,d,sales,id_str
0,30490,1942,0.997321,HOBBIES_1_001_CA_1_evaluation
1,30490,1943,0.980085,HOBBIES_1_001_CA_1_evaluation
2,30490,1944,0.998054,HOBBIES_1_001_CA_1_evaluation
3,30490,1945,0.998054,HOBBIES_1_001_CA_1_evaluation
4,30490,1946,0.998054,HOBBIES_1_001_CA_1_evaluation
...,...,...,...,...
853715,60979,1965,1.844649,FOODS_3_827_WI_3_evaluation
853716,60979,1966,1.838835,FOODS_3_827_WI_3_evaluation
853717,60979,1967,1.834941,FOODS_3_827_WI_3_evaluation
853718,60979,1968,1.834941,FOODS_3_827_WI_3_evaluation


In [ ]:
# 28 日間の売上を整形
# 1. d の範囲を制限
sales_data_filtered = sales_data[sales_data['d'].between(1942, 1942 + 27)].copy()

# 2. d を F1, F2, ..., F28 に変換
sales_data_filtered['Fn'] = 'F' + (sales_data_filtered['d'] - 1941).astype(str)
sales_data_filtered = sales_data_filtered.groupby(['id_str', 'Fn'])['sales'].mean().reset_index()

# 3. データをピボットテーブル化
pivot_data = sales_data_filtered.pivot(index='id_str', columns='Fn', values='sales').reset_index()
pivot_data=pivot_data.fillna(0)
pivot_data.rename(columns={'id_str': 'id'},inplace=True)
# pivot_data.drop(columns=['Fn'], inplace=True)
print(pivot_data)
# pivot_data.describe()


Fn                                  id        F1       F10       F11  \
0          FOODS_1_001_CA_1_evaluation  0.522065  0.571309  0.555768   
1          FOODS_1_001_CA_2_evaluation  0.614341  0.670339  0.646887   
2          FOODS_1_001_CA_3_evaluation  0.924134  0.980132  0.956680   
3          FOODS_1_001_CA_4_evaluation  0.266426  0.322424  0.306883   
4          FOODS_1_001_TX_1_evaluation  1.424009  1.475054  1.448528   
...                                ...       ...       ...       ...   
30485  HOUSEHOLD_2_516_TX_2_evaluation  0.312364  0.332320  0.316779   
30486  HOUSEHOLD_2_516_TX_3_evaluation  0.428479  0.484477  0.468936   
30487  HOUSEHOLD_2_516_WI_1_evaluation  0.192542  0.241786  0.226246   
30488  HOUSEHOLD_2_516_WI_2_evaluation  0.170871  0.213725  0.198184   
30489  HOUSEHOLD_2_516_WI_3_evaluation  0.061508  0.085426  0.077520   

Fn          F12       F13       F14       F15       F16       F17  ...  \
0      0.555768  0.532255  0.534423  0.529358  0.511344  0.52

In [ ]:
# 4. submission と結合して F1～F28 を更新
submission = submission.drop(columns=[col for col in submission.columns if col.startswith('F')])
submission_merged = pd.merge(submission,pivot_data, on='id',how='left')
submission_merged=submission_merged.fillna(0)
submission_merged

# 結果を確認
print(submission_merged)

                                  id        F1       F10       F11       F12  \
0      HOBBIES_1_001_CA_1_validation  0.000000  0.000000  0.000000  0.000000   
1      HOBBIES_1_002_CA_1_validation  0.000000  0.000000  0.000000  0.000000   
2      HOBBIES_1_003_CA_1_validation  0.000000  0.000000  0.000000  0.000000   
3      HOBBIES_1_004_CA_1_validation  0.000000  0.000000  0.000000  0.000000   
4      HOBBIES_1_005_CA_1_validation  0.000000  0.000000  0.000000  0.000000   
...                              ...       ...       ...       ...       ...   
60975    FOODS_3_823_WI_3_evaluation  0.557603  0.616559  0.601018  0.601018   
60976    FOODS_3_824_WI_3_evaluation  0.256801  0.315757  0.300216  0.300216   
60977    FOODS_3_825_WI_3_evaluation  0.716781  0.775737  0.752285  0.752285   
60978    FOODS_3_826_WI_3_evaluation  1.327429  1.458969  1.435517  1.435517   
60979    FOODS_3_827_WI_3_evaluation  1.834208  1.965749  1.942297  1.942297   

            F13       F14       F15    

## ファイル保存と送信

In [ ]:
from datetime import datetime

# 現在の日時を取得してフォーマット
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

# ファイル名に日付と時刻を追加
filename = f"submission_{timestamp}.csv"

# CSV を保存
submission_merged.to_csv(filename, index=False)

# Kaggle へ提出
!kaggle competitions submit -c m5-forecasting-accuracy -f {filename} -m "upload"
print(f"Submission saved as {filename}")


100% 20.5M/20.5M [00:02<00:00, 8.00MB/s]
Successfully submitted to M5 Forecasting - AccuracySubmission saved as submission_20250215_060155.csv
